In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [2]:
# program = "graph_coloring"
program = "maximal_matching"
# program = "dijkstra_token_ring"
# graph_name = "graph_7"
graph_name = "graph_10"

no_simulations = 100000
fault_interval = 2
sched = 0

selected_nodes = [0, 4, 7, 9]

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__duong"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,0,308.433697
1,1,307.499198
2,2,306.567526
3,3,305.638674
4,4,304.712632
...,...,...
5031,5031,-0.411146
5032,5032,-0.411146
5033,5033,-0.411146
5034,5034,-0.411147


In [5]:
dfs[0]

,Steps,exp_fit
0,0,311.508034
1,1,310.551024
2,2,309.596951
3,3,308.645806
4,4,307.697581
...,...,...
5273,5273,-0.301759
5274,5274,-0.301759
5275,5275,-0.301759
5276,5276,-0.301759


In [6]:
dfs[1]

,Steps,exp_fit
0,0,304.176420
1,1,303.268306
2,2,302.362900
3,3,301.460193
4,4,300.560177
...,...,...
5099,5099,-0.425107
5100,5100,-0.425108
5101,5101,-0.425108
5102,5102,-0.425108


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__4,exp_fit__7,exp_fit__9
Steps,,,,
0,311.508034,304.176420,308.433697,265.513557
1,310.551024,303.268306,307.499198,264.805600
2,309.596951,302.362900,306.567526,264.099530
3,308.645806,301.460193,305.638674,263.395344
4,307.697581,300.560177,304.712632,262.693036
...,...,...,...,...
5031,-0.301727,-0.425091,-0.411146,0.064892
5032,-0.301727,-0.425091,-0.411146,0.064891
5033,-0.301727,-0.425091,-0.411146,0.064890


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 4,Exponential fit of node 7,Exponential fit of node 9
Steps,,,,
0,311.508034,304.176420,308.433697,265.513557
1,310.551024,303.268306,307.499198,264.805600
2,309.596951,302.362900,306.567526,264.099530
3,308.645806,301.460193,305.638674,263.395344
4,307.697581,300.560177,304.712632,262.693036
...,...,...,...,...
5031,-0.301727,-0.425091,-0.411146,0.064892
5032,-0.301727,-0.425091,-0.411146,0.064891
5033,-0.301727,-0.425091,-0.411146,0.064890


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [11]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}__{fault_interval}.png",
    bbox_inches="tight",
)
plt.close()